In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
from sklearn.naive_bayes import GaussianNB, ComplementNB, CategoricalNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split # for train-test split 
from sklearn.preprocessing import StandardScaler # for feature scaling
from sklearn.model_selection import GridSearchCV # for fine-tuning
from sklearn.metrics import classification_report, confusion_matrix, plot_roc_curve # for evaluation
from sklearn.pipeline import make_pipeline # for prediction

In [ ]:
# for Generator
from scipy import stats # for sampling
from fitter import Fitter # for fitting the best distribution
import copy # for copying nested dictionaries

In [ ]:
import matplotlib.pyplot as plt  # for visualization 
import seaborn as sns  # for coloring 

# set style of graphs
plt.style.use('seaborn')
from pylab import rcParams
plt.rcParams['figure.dpi'] = 80

In [ ]:
train_data = pd.read_csv('../dataset/train_data.csv')

In [ ]:
feature_list = list(train_data.columns)
feature_list

In [ ]:
#✍To predict win/loss of a game, we can use one of the two ways:

#1. Select only one feature (points), the win/loss prediction is just based on which team has the higher point.
#2. Select features other than points, the win/loss is then based on the prediction of a classifier which takes those features as inputs.

# In this notebook, we will use option (2) as it offers better range of uncertainty for simulation.

selected_features = [
    'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home',
    'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away',
    ]

# check the features we selected
X = train_data[selected_features]
X.head()

In [ ]:
y = train_data['HOME_TEAM_WINS']
y.head()

In [ ]:
X = X.to_numpy()
y = y.to_numpy()

In [ ]:
# splitting 70% training data, 30% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("X shape", X_train.shape, "y shape", y_train.shape)

##### Utility function

In [ ]:
def evaluate(y_test, y_pred):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    # print(f"accuracy: {accuracy_score(y_test, y_pred)}")
    # print(f"AUC: {roc_auc_score(y_test, y_pred)}")
    # print(f"log_loss: {log_loss(y_test, y_pred)}")

## 1. SVM

In [ ]:
# for SVM a scaler it can improve accuracy of the model

# scaler = StandardScaler() # initialize an instance 
# X_train_svm = scaler.fit_transform(X_train)

In [ ]:
%%time 

# train SVM

support_vector_default = SVC() # initialize a model
support_vector_default.fit(X_train, y_train) # fit(train) it with the training data and targets

# check test score 
y_pred_svm_default = support_vector_default.predict(X_test) 

#### Grid search SVM

In [ ]:
%%time 

# fine-tuning hyperparameters
#param_grid_svm = {'C': [0.1, 1, 10],
#              'gamma': [1, 0.5, 0.1, 0.01, 0.001, 0.0001],
#              'kernel': ['linear', 'poly', 'sigmoid', 'rbf']
#            }
param_grid_svm = {'C': [0.1],
              'gamma': [0.1],
              'kernel': ['linear', 'poly', 'rbf']
            }

grid_search_svm = GridSearchCV(estimator=support_vector_default, param_grid=param_grid_svm, cv=10, verbose=2, scoring='accuracy', 
                            n_jobs = -1, return_train_score=True)

In [ ]:
# grid_search_svm.fit(X_train, y_train)

In [ ]:
# grid_search_svm.best_params_

# {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}

In [ ]:
%%time 

# train SVM

support_vector = SVC(C=0.1, gamma=0.1, kernel='linear') # initialize a model
support_vector.fit(X_train, y_train) # fit(train) it with the training data and targets

# check test score 
y_pred_svm = support_vector.predict(X_test) 

## 2. Random Forest

In [ ]:
#Create a Gaussian Classifier
random_forest_default = RandomForestClassifier()

# clf = RandomForestClassifier(n_estimators=20, random_state=0)

#Train the model using the training sets y_pred=clf.predict(X_test)
random_forest_default.fit(X_train,y_train)

y_pred_rf_default = random_forest_default.predict(X_test)

#### Grid Search Random Forest

In [ ]:
# another round

param_grid_rf = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [10, 20, 30, 40, 50],
    'bootstrap': [True, False]
    }

# Create a base model
grid_search_rf = GridSearchCV(estimator=random_forest_default, param_grid=param_grid_rf, cv=10, verbose=2, scoring='accuracy', 
                            n_jobs = -1, return_train_score=True)

In [ ]:
# grid_search_rf.fit(X_train, y_train)

In [ ]:
# grid_search_rf.best_params_

# {'bootstrap = True', max_depth = 30, max_features = 'auto', min_samples_leaf = 2, min_samples_split = 2, n_estimators = 400}

In [ ]:
#Create a Gaussian Classifier
# random_forest = RandomForestClassifier(n_estimators=400, min_samples_split=2, min_samples_leaf=2, max_features='auto', max_depth=30, bootstrap='True')
random_forest = RandomForestClassifier(n_estimators=500, max_features='sqrt', max_depth=50, bootstrap='True')
# clf = RandomForestClassifier(n_estimators=20, random_state=0)

#Train the model using the training sets y_pred=clf.predict(X_test)
random_forest.fit(X_train,y_train)

y_pred_rf = random_forest.predict(X_test)

## 3. Naive Bayes

In [ ]:
gauss_nb_default = GaussianNB()
gauss_nb_default.fit(X_train, y_train)
y_pred_gauss_nb_default = gauss_nb_default.predict(X_test)

#### Grid search Naive Bayes

In [ ]:
param_grid_nb = {
    'var_smoothing': np.logspace(0,-9, num=100)
    }

In [ ]:
grid_search_nb = GridSearchCV(estimator=gauss_nb_default, param_grid=param_grid_nb, cv=10, verbose=2, scoring='accuracy', 
                            n_jobs = -1, return_train_score=True)

In [ ]:
# grid_search_nb.fit(X_train, y_train)

In [ ]:
# grid_search_nb.best_params_

In [ ]:
gauss_nb = GaussianNB(var_smoothing=1.873817422860383e-07)
gauss_nb.fit(X_train, y_train)
y_pred_gauss_nb = gauss_nb.predict(X_test)

### Metrics

In [ ]:
# Plotting AUC for final hyperparametrizes classifiers

fig = plot_roc_curve(support_vector, X_test, y_test, linewidth=5, linestyle = '-')
fig = plot_roc_curve(gauss_nb, X_test, y_test, ax = fig.ax_, linewidth=5, linestyle = ':')
fig = plot_roc_curve(random_forest, X_test, y_test, ax = fig.ax_, linewidth=5, linestyle = '--')

plt.tick_params(axis='x', labelsize=24)
plt.tick_params(axis='y', labelsize=24)
plt.xlabel('False Positive Rate', fontsize=24)
plt.ylabel('True Positive Rate', fontsize=24)

plt.legend(fontsize=24) # using a size in points
plt.legend(fontsize="xx-large") # using a named size

plt.savefig('../report/plots/auc_optimize.png')

In [ ]:
# plotting AUC for standard classifier

fig = plot_roc_curve(support_vector_default, X_test, y_test, linewidth=3.5, linestyle = '-')
fig = plot_roc_curve(gauss_nb_default, X_test, y_test, ax = fig.ax_, linewidth=3.5, linestyle = ':')
fig = plot_roc_curve(random_forest_default, X_test, y_test, ax = fig.ax_, linewidth=3.5, linestyle = '--')

plt.tick_params(axis='x', labelsize=24)
plt.tick_params(axis='y', labelsize=24)
plt.xlabel('False Positive Rate', fontsize=24)
plt.ylabel('True Positive Rate', fontsize=24)

plt.legend(fontsize=24) # using a size in points
plt.legend(fontsize="xx-large") # using a named size

plt.savefig('../report/plots/auc_default.png')

In [ ]:
# metrics evaluation for standard classifier

evaluate(y_test, y_pred_rf_default)
evaluate(y_test, y_pred_svm_default)
evaluate(y_test, y_pred_gauss_nb_default)

In [ ]:
# metrics evaluation for hyperparametrize classifier

evaluate(y_test, y_pred_rf)
evaluate(y_test, y_pred_svm)
evaluate(y_test, y_pred_gauss_nb)

### 3. Fitting a Generator

In [ ]:
# Like before, we had held out data from 2020-2021 playoff for real testing
# Though large data is essential for fitting, for time-series problems, we give priority to the recent data most reflective of team's recent ability.
# Since we aim to predict 2021-2022 playoff, here we will just fit the data from that regular session which starts in Oct, 2020.

df_ = train_data.loc[train_data['SEASON'] > 2019].reset_index(drop=True)

In [ ]:
selected_distributions = [
    'norm','t', 'f', 'chi', 'cosine', 'alpha', 
    'beta', 'gamma', 'dgamma', 'dweibull',
    'maxwell', 'pareto', 'fisk'
    ]

In [ ]:
unique_teams = train_data['HOME_TEAM_ID'].unique() # extract all the unique teams

# Since we don't care about whether the team was a host or visitor in each game, 
# we can just combine the features for all games.

# Get all the data for teams
all_team_sim_data = {}

for team_name in unique_teams:
    
    # find games where the team is either the host or guest
    df_team = df_.loc[(df_['HOME_TEAM_ID'] == team_name) | (df_['VISITOR_TEAM_ID'] == team_name)]
    # it is home team, select the first 5 features
    df_1 = df_team.loc[df_team['HOME_TEAM_ID'] == team_name][selected_features[:5]]
    # it is guest team, select the first 5 features
    df_0 = df_team.loc[df_team['VISITOR_TEAM_ID'] == team_name][selected_features[5:]]

    # combine them
    df_0.columns = df_1.columns # before concating, match the column names
    df_s = pd.concat([df_1, df_0], axis = 0)
    
    # convert the pandas.DataFrame to numpy array
    all_team_sim_data[team_name] = df_s.to_numpy()


In [ ]:
all_team_sim_data

In [ ]:
# data format:
#   team_name => list of feature distributions => dictionary with distribution name and parameters
#   e.g.,
#   megadata = {
      #'Timberwolves': [{'beta': (0.23, 0.3, 0.3, 0.4)}, {'nor': (0.23, 0.3,)}, ..], 
      #'Warriors':[{}, {},...]
      #  }
    
megadata = {} # store the data that our Generator will rely on
for team_name in unique_teams:
    
    feature_dis_paras = []
    data = all_team_sim_data[team_name]
    
    # 5 features for each team
    for i in range(5): 
        f = Fitter(data[:, i]) # initalize a Fitter instance
        f.distributions = selected_distributions # use only the selected distributions (faster)
        f.fit() # do the fitting 
        best_paras = f.get_best(method='sumsquare_error') # get the best fitted paras
        feature_dis_paras.append(best_paras)
        
    megadata[team_name] = feature_dis_paras
    
# print('Features for all teams have been fitted!')

### 4. Simulation

In [ ]:
DATA = megadata.copy() # data that Generator must rely on

GEN = {
 'alpha': stats.alpha.rvs,
 'beta': stats.beta.rvs,
 'chi': stats.chi.rvs,
 'cosine': stats.cosine.rvs,
 'dgamma': stats.dgamma.rvs,
 'dweibull':stats.dweibull.rvs,
 'f':stats.f.rvs,
 'fisk':stats.fisk.rvs,
 'gamma': stats.gamma.rvs,
 'maxwell':stats.maxwell.rvs,
 'norm':stats.norm.rvs,
 'pareto':stats.pareto.rvs,
 't':stats.t.rvs,
}

In [ ]:
# DIS = make_pipeline(scaler, support_vector)
# DIS = make_pipeline(random_forest)
DIS = make_pipeline(gauss_nb)

Process: 

1. sampling: "generate feature values used for making win/loss prediction"
2. predict: "predict the win or loss of  n game(s) played by two tems"

In [ ]:
class Game:
    
    '''
    
    A game between two teams:
    
    - feature values sampled from Generator
    - win/loss predicted by Discriminator
    
    '''
    
    def __init__ (self, random_state = None):
        
        self.random_state = random_state # keep this to None for making simulations 
    
    def predict(self, team1, team2, num_games = 1):
        
        '''Taking as input two teams predict the win or loss of  n game(s)'''
        
        assert num_games >= 1, "at least one game must be played"
        # output numpy array
        team_1_feature_data = DATA[team1] # take generator data from team 1
        team_2_feature_data = DATA[team2] # take generator data from team 2
        
        features = []
        for feature_paras_1 in team_1_feature_data:
            # sampling 5 features for team 1
            sample_1 = self.sampling(feature_paras_1, size = num_games) 
            features.append(sample_1) 
            
        for feature_paras_2 in team_2_feature_data:
            # sampling 5 features for team 1
            sample_2 = self.sampling(feature_paras_2, size = num_games) 
            features.append(sample_2)

        features = np.array(features).T # collect together the features of the two teams 
                                        # to simulate a possible match

        win_loss = DIS.predict(features) # use the classifier to predict the result of this hypotethical match
        
        return list(win_loss) # a list of win/loss from num_games
    
    
    def sampling(self, dic, size = 1, random_state = None):
        
        '''generate feature values used for making win/loss prediction'''
                        
        dis_name = list(dic.keys())[0] # get the distribution type
        paras = list(dic.values())[0] # get the distribution parameters
    
        # create the random sample from the distribution
        sample = GEN[dis_name](*paras, size = size,  random_state =  random_state)
        
        return sample

In [ ]:
class FinalTournament(Game):
    
    ''' Best-of-7 elimination, 16 teams, 4 rounds in total to win championship '''
    
    def __init__(self, n_games_per_group = 7, winning_threshold = 4, random_state = None):

        self.n_games_per_group  = n_games_per_group
        self.winning_threshold = winning_threshold
        self.team_list = None
        self.rounds = {} # keep track the number of times a team wins at each round 
        super().__init__(random_state)
        
    
    def simulate(self, group_list, n_simulation = 1, probs = True):
        
        ''' simulate the entire playoff n times and also record the accumulated wins'''
             
        # update the list of teams
        self.rounds = {}
        self.team_list = [i[0] for i in group_list] + [i[1] for i in group_list]
        
        for i in range(n_simulation):
            # print(f"epoch number: {i}")
            cham = self.one_time_simu(group_list)
        if probs:
            self.rounds_probs = self._compute_probs()
            
    
    def one_time_simu(self, group_list, verbose = False, probs = False):
        
        ''' simulate the entire playoff once and also record the accumulated wins'''
        
        # update the list of teams if haven't done so
        if self.team_list == None: 
            self.team_list = [i[0] for i in group_list] + [i[1] for i in group_list]
        round_number, done = 0, 0
        while not done: 
            all_group_winners, group_list = self.play_round(group_list)
            # retrive round stats
            try:
                updated_round_stats = self.rounds[round_number]
            except KeyError:
                updated_round_stats = {}
                for team in self.team_list:
                    updated_round_stats[team] = 0
            # if a team wins, record + 1 
            for winner in all_group_winners:
                try: 
                    updated_round_stats[winner] += 1
                except KeyError:
                    pass     
            self.rounds[round_number] = updated_round_stats
            if verbose:
                print('{} round played'.format(round_number))
            if probs:
                self.rounds_probs = self._compute_probs()
            if type(group_list) != list: # if it becomes the final
                done = 1
            round_number += 1
            
        return group_list

        
    def play_round(self, group_list):
        
        '''play a round of games based of a list of paired teams'''
        
        all_group_winners = [] 
        # play each group and get the group winner
        for group in group_list:
            winner = self.play_n_games(group[0], group[1])
            all_group_winners.append(winner)
        
        if len(all_group_winners) > 1:
            new_group_list = []         
            for index in range(0, len(all_group_winners), 2):
                # first winner, second winner
                new_group = [all_group_winners[index], all_group_winners[index + 1]]
                new_group_list.append(new_group)
                
            return all_group_winners, new_group_list
        else:  
            return all_group_winners, winner
        
        
    def play_n_games(self, team1, team2):
        
        
        '''simulate data, and then use our classifier to predict win/loss'''
        result = Game().predict(team1, team2, self.n_games_per_group)
        if sum(result[:4]) == self.winning_threshold or sum(result) >= self.winning_threshold:
            winner = team1 # home team wins
        else:
            winner = team2 # visitor team wins
            
        return winner
    
    def rounds_list(self):
        list_round = self.rounds
        return list
    
    def _compute_probs(self):
        
        '''prob = wins for a team / sum of wins for all teams at a particular round'''
        
        rounds_probs = copy.deepcopy(self.rounds)
        for round_number, round_stats in rounds_probs.items():
            m = np.sum(list(round_stats.values()))
            for k, v in rounds_probs[round_number].items():
                rounds_probs[round_number][k] = v / m
                
        return rounds_probs

In [ ]:
#2022
group_list_2022 = [
     # Eastern Conference
     ('Heat', 'Hawks'),  # group A 1 
     ('76ers', 'Raptors'), # group B 4 
    
     ('Bucks', 'Bulls'), # group C 3 
     ('Celtics', 'Nets'), # group D 2
    
     # Western Conference
     ('Suns','Pelicans'),  # group E 1 
     ('Mavericks','Jazz'), # group F 4 
    
     ('Warriors', 'Nuggets'), # group G 3 
     ('Grizzlies', 'Timberwolves')] # group H 2

In [ ]:
%%time

# initiate a playoff
playoff = FinalTournament()

# simulate the playoff 5,000 times

playoff.simulate(group_list_2022, n_simulation = 5000)


In [ ]:
playoff.rounds_probs

### 5. Visualization & Analysis

In [ ]:
def plotting(rounds_data):
    
    rounds_stats = list(rounds_data.values())
    team_names = list(rounds_stats[0].keys())
    
    # x is number of rounds used for labels, y is a 2-D array of (n_teams, n_rounds) used for data
    x = list(rounds_data.keys())
    y = np.array([list(r.values()) for r in rounds_stats]).T 
    
    # we need at least 16 different colors, one for each team
    c_1 =  sns.color_palette('tab10', n_colors = 10)
    c_2 =  sns.color_palette("pastel", n_colors = 10)
    color_map = c_1 + c_2 
    
    fig = plt.figure()
    plt.stackplot(x, y, labels = team_names, colors = color_map) 
    plt.legend(bbox_to_anchor=(1.1, 1.1), loc = 'upper left', fontsize=13)
    plt.xticks(x, fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel('Round Number', fontsize = 15)
    plt.title('Winning probabilities by all Teams & Rounds', pad = 20, fontsize = 24)
    plt.tight_layout()
    plt.show()
    
    return fig

In [ ]:
# check that a team's wins should get less and less in later rounds
fig = plotting(playoff.rounds)

In [ ]:
# plot the results: probabilities of winning for all teams at each round
fig = plotting(playoff.rounds_probs)